In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Path to your dataset
data_path = os.path.join('..', '..', 'data', 'processed', 'dl_ready_flights', 'dl_tabular_data.csv')

# Load dataset
df = pd.read_csv(data_path)

print("✅ Loaded data shape:", df.shape)
df.head()


✅ Loaded data shape: (600000, 53)


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DAY_OF_WEEK_NORM,DEP_HOUR_NORM,MONTH_SIN,MONTH_COS,DAY_OF_WEEK_SIN,DAY_OF_WEEK_COS,DEP_HOUR_SIN,DEP_HOUR_COS,ORIGIN_FLIGHTS,ORIGIN_FLIGHTS_NORM
0,2021-07-25,0.944530,0.944530,1.067426,-1.552522,2.460211,-0.842944,-0.835750,-0.756873,-0.819571,...,1.513621,0.817800,-0.733237,-1.111240,-1.134983,0.871194,-1.181304,0.199712,1.468965,1.468965
1,2019-11-09,-1.266437,-1.266437,-1.397812,-0.453101,-1.314106,-1.059691,-1.101914,1.462173,1.218245,...,1.013834,-1.234219,-0.733237,1.341978,-1.411967,-0.308945,1.383718,0.199712,0.867214,0.867214
2,2023-05-26,0.944530,0.944530,1.067426,-1.552522,-0.453622,-1.049839,-0.940314,-1.585316,-1.495669,...,0.514047,-1.029017,0.688621,-1.111240,-0.635874,-1.255343,1.251075,-0.246307,-0.859627,-0.859627
3,2023-03-23,0.774456,0.774456,0.539160,0.878478,0.530202,-0.567084,-0.512550,0.840840,0.913525,...,0.014260,-1.439421,1.399550,0.115369,0.608882,-1.255343,1.428960,0.678350,-1.143316,-1.143316
4,2022-03-27,1.284679,1.284679,0.891337,0.005880,-1.048896,-0.606493,0.599636,1.264924,1.180155,...,1.513621,-0.208209,1.399550,0.115369,-1.134983,0.871194,0.101207,-1.170964,-0.024023,-0.024023


In [ ]:
# Convert DEP_DELAY to numeric (force bad values to NaN)
df['DEP_DELAY'] = pd.to_numeric(df['DEP_DELAY'], errors='coerce')

# Show after fix
print(df['DEP_DELAY'].describe())
print("NaNs in DEP_DELAY after fix:", df['DEP_DELAY'].isna().sum())


count    600000.000000
mean          0.009300
std           1.091749
min          -0.294091
25%          -0.294091
50%          -0.294091
75%          -0.132652
max          53.989702
Name: DEP_DELAY, dtype: float64
NaNs in DEP_DELAY after fix: 0


In [ ]:
# First convert to datetime temporarily
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')

# Extract numeric date parts
df['FL_MONTH'] = df['FL_DATE'].dt.month
df['FL_DAY'] = df['FL_DATE'].dt.day
df['FL_DOW'] = df['FL_DATE'].dt.dayofweek

# Drop the original datetime column
df = df.drop(columns=['FL_DATE'])


In [ ]:
# ✅ Remove 'CANCELLATION_CODE' column if it exists
if 'CANCELLATION_CODE' in df.columns:
    df = df.drop(columns=['CANCELLATION_CODE'])
    print("✅ 'CANCELLATION_CODE' column removed.")
else:
    print("ℹ️ 'CANCELLATION_CODE' column not found — nothing to remove.")


✅ 'CANCELLATION_CODE' column removed.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset

# Define target and features
target = 'DEP_DELAY'
X = df.drop(columns=[target, 'FL_DATE'], errors='ignore')
y = df[target]

# Keep only numeric features
X = X.select_dtypes(include=[np.number])

# Train/val/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize inputs
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from rtdl import FTTransformer
from torch import nn

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=512, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=512)

# Define model
model = FTTransformer(
    d_numerical=X_train.shape[1],
    categories=[],
    token_bias=True,
    n_layers=3,
    n_heads=4,
    d_token=64,
    attention_dropout=0.1,
    ffn_dropout=0.1,
    residual_dropout=0.1
)
model = nn.Sequential(model, nn.Linear(64, 1))  # Regression head


ModuleNotFoundError: No module named 'rtdl'

In [ ]:
import numpy as np

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_mae, val_mae = [], []

for epoch in range(10):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).squeeze().numpy()
        val_pred = model(X_val).squeeze().numpy()
        train_mae.append(np.mean(np.abs(train_pred - y_train.numpy().squeeze())))
        val_mae.append(np.mean(np.abs(val_pred - y_val.numpy().squeeze())))
    print(f"Epoch {epoch+1}: Train MAE={train_mae[-1]:.4f}, Val MAE={val_mae[-1]:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_mae, label="Train MAE")
plt.plot(val_mae, label="Val MAE")
plt.title("FT-Transformer: MAE Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
with torch.no_grad():
    y_pred = model(X_test).squeeze().numpy()

plt.figure(figsize=(6, 6))
plt.scatter(y_test.numpy(), y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual DEP_DELAY")
plt.ylabel("Predicted DEP_DELAY")
plt.title("FT-Transformer: Actual vs Predicted")
plt.grid(True)
plt.show()


NameError: name 'model' is not defined